In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [19]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount2.csv')
df_test = pd.read_csv('datasets/SatisfactionClients/amazon_test1.csv')
df.sample(3)

,note,commentaire
74622,2,commande très difficile annuler cela n'est con...
23896,5,tres bien ravie cet achat
4468,4,satisfaite discount partenaires comme trustpil...


In [20]:
df.isna().sum()

note            0
commentaire    54
dtype: int64

In [21]:
df[df['commentaire'].isna()==True]

,note,commentaire
156,5,NaN
364,4,NaN
473,5,NaN
476,5,NaN
1459,5,NaN
2040,3,NaN
2144,5,NaN
2293,5,NaN
2749,5,NaN
2920,4,NaN


In [22]:
df = df[df['commentaire'].isna()==False]

In [23]:
# séparation de la variable cible et des variables explicatives
X = df['commentaire']
y = df['note']

In [24]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [25]:
# conversion des chaines de caratères en tokens numériques
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, ngram_range=[1, 2])
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [26]:
np.save('datasets/SatisfactionClients/vector_matrix_ngram_1_2', X_train)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=50)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test)

for i in cm_test.index:
    for j in cm_test.columns :
        if (i==j and cm_test.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6798910468972051

              precision    recall  f1-score   support

           1       0.71      0.78      0.74      2004
           2       0.13      0.01      0.02       505
           3       0.35      0.13      0.19      1135
           4       0.40      0.25      0.31      3436
           5       0.74      0.91      0.81      9808

    accuracy                           0.68     16888
   macro avg       0.46      0.42      0.41     16888
weighted avg       0.62      0.68      0.64     16888



predictions,1,2,3,4,5
données réelles,,,,,
1,1573,10,68,104,249
2,210,5,69,75,146
3,223,15,151,328,418
4,137,5,103,853,2338
5,87,3,44,774,8900


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,3,4,5
données réelles,,,,
1,18,0,2,0
2,17,0,2,1
3,13,2,4,1
4,10,0,2,8
5,8,0,0,12


predictions,1,3,4,5
données réelles,,,,
1,0.90,0.0,0.1,0.00
2,0.85,0.0,0.1,0.05
3,0.65,0.1,0.2,0.05
4,0.50,0.0,0.1,0.40
5,0.40,0.0,0.0,0.60


les prédictions correctes sont supérieures à 50% pour 1 étoile(s) avec 90.0 %
les prédictions correctes sont supérieures à 50% pour 5 étoile(s) avec 60.0 %


In [28]:
from joblib import dump
dump(clf, 'datasets/SatisfactionClients/model_rf_ngrams_1_2.joblib')

['datasets/SatisfactionClients/model_rf_ngrams_1_2.joblib']

### Evaluation sur 2 sentiments : négatif et positif

* Le DataSet cdiscount a été retraité en renommant, les notes 1 et 2 en 0, les notes 4 et 5 en 1 et en supprimant les notes 3
* Le but étant d'isoler de manière moins complexe les sentiments négatifs et les sentiments positifs et ainsi améliorer la qualité du résultat prédit

In [29]:
df_0_1 = pd.read_csv('datasets/SatisfactionClients/cdiscount_0_1.csv')

In [30]:
df_0_1.isna().sum()

note            0
commentaire    46
dtype: int64

In [31]:
df_0_1 = df_0_1[df_0_1['commentaire'].isna()==False]

In [32]:
# séparation de la variable cible et des variables explicatives
X1 = df_0_1['commentaire']
y1 = df_0_1['note']

In [33]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, shuffle=True)

In [34]:
X_train1 = vectorizer.transform(X_train1).todense()
X_test1 = vectorizer.transform(X_test1).todense()

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf_0_1 = RandomForestClassifier(n_estimators=50)

clf_0_1.fit(X_train1, y_train1)

y_pred_0_1 = clf_0_1.predict(X_test1)

print(clf_0_1.score(X_test1, y_test1), end='\n\n')

cr = classification_report(y_test1, y_pred_0_1)
print(cr)

cm = pd.crosstab(y_test1, y_pred_0_1, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf_0_1.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.9443140736982305

              precision    recall  f1-score   support

           0       0.87      0.76      0.81      2489
           1       0.96      0.98      0.97     13278

    accuracy                           0.94     15767
   macro avg       0.91      0.87      0.89     15767
weighted avg       0.94      0.94      0.94     15767



predictions,0,1
données réelles,,
0,1882,607
1,271,13007


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,0,1
données réelles,,
1,16,4
2,18,2
3,12,8
4,10,10
5,8,12


predictions,0,1
données réelles,,
1,0.8,0.2
2,0.9,0.1
3,0.6,0.4
4,0.5,0.5
5,0.4,0.6


In [36]:
from joblib import dump
dump(clf_0_1, 'datasets/SatisfactionClients/model_rf_0_1_ngrams_1_2.joblib')

['datasets/SatisfactionClients/model_rf_0_1_ngrams_1_2.joblib']